In [ ]:
%matplotlib notebook
import matplotlib.pyplot as plt
from astropy.table import Table
import numpy as np
import scipy.optimize as op
import os
import sys
sys.path.append(f'{os.environ["HOME"]}/Projects/planckClusters/catalogs')
from load_catalogs import load_PSZcatalog
from tqdm import tqdm_notebook
                
import warnings
from astropy.utils.exceptions import AstropyWarning
warnings.simplefilter('ignore', category=AstropyWarning)

In [ ]:
from concurrent.futures import ThreadPoolExecutor, as_completed

def parallel_process(array, function, n_jobs=None, use_kwargs=False, front_num=0):
    """
        A parallel version of the map function with a progress bar. 

        Args:
            array (array-like): An array to iterate over.
            function (function): A python function to apply to the elements of array
            n_jobs (int, default=16): The number of cores to use
            use_kwargs (boolean, default=False): Whether to consider the elements of array as dictionaries of 
                keyword arguments to function 
            front_num (int, default=3): The number of iterations to run serially before kicking off the 
                parallel job. This can be useful for catching bugs
        Returns:
            [function(array[0]), function(array[1]), ...]
    """
    #We run the first few iterations serially to catch bugs
    if front_num > 0:
        front = [function(**a) if use_kwargs else function(a) for a in array[:front_num]]
    #If we set n_jobs to 1, just run a list comprehension. This is useful for benchmarking and debugging.
    if n_jobs==1:
        [function(**a) if use_kwargs else function(a) for a in tqdm_notebook(array[front_num:])]
        return 
    #Assemble the workers
    with ThreadPoolExecutor(max_workers=n_jobs) as pool:
        #Pass the elements of array into function
        if use_kwargs:
            futures = [pool.submit(function, **a) for a in array[front_num:]]
        else:
            futures = [pool.submit(function, a) for a in array[front_num:]]
        kwargs = {
            'total': len(futures),
            'unit': 'it',
            'unit_scale': False,
            'leave': True
        }
        #Print out the progress as tasks complete
        for f in tqdm_notebook(as_completed(futures), **kwargs):
            pass

In [ ]:
def beta_model(So, r, rc, beta):
    ''' Beta model with 3 parameters. 
    So -- normalization
    rc -- core radius
    beta -- powerlaw slope

    '''
    
    return So * ( 1.0 + (r / rc)**2)**(-3.0 * beta + 0.5)

In [ ]:
def chi2(model, y, y_err):
    '''Chi error. We are going to square this to make it the chi2 error.'''
    return np.sum(((model - y) / y_err)**2)

In [ ]:
def like(theta, x, y, yerr):
    So, rc, beta, bg = theta
    model = beta_model(So, x, rc, beta) + bg
    return chi2(model, y, yerr)

def prior(theta):
    So, rc, beta, bg = theta
    if So < 0:
        return np.inf
    elif rc < 0:
        return np.inf
    elif beta < 0 or beta > 3: 
        return np.inf
    elif bg < 0:
        return np.inf
    else:
        return 0.0

def prob(theta, x, y, yerr):
    lp = prior(theta)
    if not np.isfinite(lp):
        return -np.inf
    return -lp + -like(theta, x, y, yerr)

In [ ]:
def fit_profile(name, outpath):
    
    # swift pixelscale in degrees
    pixsc =  6.548089E-04 * 3600
    
    if os.path.isfile(f'{outpath}/{name}/{name}_vtp.detect'):
        srcs = f'{outpath}/{name}/{name}_vtp.detect'
        # now we need to read the individual detections
        detects = Table.read(srcs, hdu=1)
    else:
        return
    
    # we are going to fit profiles to the top 3 biggest 
    detects.sort(['SRC_AREA'], reverse=True)
    
    # loop over the sources -- only the first 3
    for i in detects['COMPONENT'][:3]:
        
        if os.path.isfile(f'{outpath}/{name}/{name}_vtp_{i-1}.radprof'):
            data = Table.read(f'{outpath}/{name}/{name}_vtp_{i-1}.radprof', format='ascii', header_start=2)
        else:
            continue
            
        # x-axis, in arcminutes
        x = (data['r1'] + data['r2'])/ 2. / 60. * pixsc
        
        # this is the parameter fitting
        nll = lambda *args: prob(*args)
        result = op.minimize(nll, [0.001, 1, 1, 0.001], args=(x, data['sb'], data['sb_err']))
        
        
        # plot the result
        fig, ax = plt.subplots(ncols=1, nrows=1, figsize=(6.5, 5))
        # data
        ax.errorbar(x, data['sb'], yerr=data['sb_err'], fmt='o', label='data')
        ylims = ax.get_ylim()
        
        #fit
        So, rc, beta, bg = result['x']
        ax.plot(x, beta_model(So, x, rc, beta) + bg, label=r'$\beta$ + bg')
        ax.plot(x, beta_model(So, x, rc, beta), label=r'$\beta$')
        
        # rc and bg lines
        ax.axhline(bg, label='bg', zorder=0, lw=1)
        ax.text(rc + 0.1, 2e-5, f"rc = {rc:.2f}'", rotation='vertical', ha='left')
        ax.axvline(rc, zorder=0, lw=1)
        
        ax.semilogy()

        if ylims[0] < 1e-5:
            ax.set_ylim(1e-5, ylims[1])
        else:
            ax.set_ylim(ylims)
        ax.set_xlabel('Radius [arcmin]')
        ax.set_ylabel('Flux [cnts/s]')
        ax.legend(loc='upper right')
        
        fig.savefig(f'{outpath}/{name}/{name}_radproffit_{i-1}.png', bbox='tight', dpi=180)
        plt.close()
        

In [ ]:
# get file data
data = load_PSZcatalog()
data = data.sort_index(axis=1)

outpath = './data_full'

arr = [{'name':n.replace(' ', '_'), 'outpath':outpath} for n in data['NAME']]
#parallel_process(arr, fit_profile, use_kwargs=True, n_jobs=1)

In [ ]:
d = Table.read('./data_full/PSZ2_G018.92-33.64/PSZ2_G018.92-33.64_vtp_0.radprof', format='ascii', header_start=2)
#d = Table.read('./data_full/PSZ1_G083.35+76.41/PSZ1_G083.35+76.41_vtp_7.radprof', format='ascii', header_start=2)
d = Table.read('./data_full/PSZ1_G057.42-10.77/PSZ1_G057.42-10.77_vtp_2.radprof', format='ascii', header_start=2)
d = Table.read('./data_full/PSZ2_G356.21-43.11/PSZ2_G356.21-43.11_vtp_0.radprof', format='ascii', header_start=2)
d = Table.read('./data_full/PSZ2_G003.21-76.04/PSZ2_G003.21-76.04_vtp_1.radprof', format='ascii', header_start=2)
d = Table.read('./data_full/PSZ2_G028.08+10.79/PSZ2_G028.08+10.79_vtp_0.radprof', format='ascii', header_start=2)

#PSZ2_G028.08+10.79


In [ ]:
# swift pixelscale in degrees
pixsc =  6.548089E-04 * 3600
x = (d['r1'] + d['r2'])/ 2. / 60. * pixsc
nll = lambda *args: prob(*args)
result = op.minimize(nll, [0.001, 1, 1, 0.001], args=(x, d['sb'], d['sb_err']))

In [ ]:
fig, ax = plt.subplots(ncols=1, nrows=1, figsize=(6.5, 5))
# data
ax.errorbar(x, d['sb'], yerr=d['sb_err'], fmt='o', label='data')

ylims = ax.get_ylim()

#fit
So, rc, beta, bg = result['x']
ax.plot(x, beta_model(So, x, rc, beta) + bg, label=r'$\beta$ + bg')
ax.plot(x, beta_model(So, x, rc, beta), label=r'$\beta$')

ax.axhline(bg, label='bg', zorder=0, lw=1)

ax.text(rc + 0.1, 2e-5, f"rc = {rc:.2f}'", rotation='vertical', ha='left')
ax.axvline(rc, zorder=0, lw=1)
ax.semilogy()

if ylims[0] < 1e-5:
    ax.set_ylim(1e-5, ylims[1])
else:
    ax.set_ylim(ylims)
ax.set_xlabel('Radius [arcmin]')
ax.set_ylabel('Flux [cnts/s]')
ax.legend(loc='upper right')

In [ ]:
result

In [ ]:
ndim, nwalkers = 4, 100
pos = [result["x"] + 1e-4*np.random.randn(ndim) for i in range(nwalkers)]

In [ ]:
import emcee
sampler = emcee.EnsembleSampler(nwalkers, ndim, prob, args=(x, d['sb'], d['sb_err']))

In [ ]:
sampler.run_mcmc(pos, 500)

In [ ]:
samples = sampler.chain[:, 50:, :].reshape((-1, ndim))

In [ ]:
samples.shape

In [ ]:
So, rc, beta, bg = map(lambda v: (v[1], v[2]-v[1], v[1]-v[0]),
                             zip(*np.percentile(samples, [16, 50, 84],
                                                axis=0)))

In [ ]:
print(So)
print(rc)
print(beta)
print(bg)

In [ ]:
So, rc, beta, bg = samples[:4]

In [ ]:
np.median(So)

In [ ]:
fig, ax = plt.subplots(ncols=1, nrows=1, figsize=(6.5, 5))
# data
ax.errorbar(x, d['sb'], yerr=d['sb_err'], fmt='o', label='data')
for So, rc, beta, bg in samples[np.random.randint(len(samples), size=100)]:
    ax.plot(x, beta_model(So, x, rc, beta) + bg, color='k', alpha=0.1)
ylims = ax.get_ylim()

#fit
# So, rc, beta, bg = result['x']


So, rc, beta, bg = map(lambda v: (v[1], v[2]-v[1], v[1]-v[0]),
                             zip(*np.percentile(samples, [16, 50, 84],
                                                axis=0)))

ax.plot(x, beta_model(So[0], x, rc[0], beta[0]) + bg[0], label=r'$\beta$ + bg')
ax.plot(x, beta_model(So[0], x, rc[0], beta[0]), label=r'$\beta$')


ax.axhline(bg[0], label='bg', zorder=0, lw=1)

ax.text(rc[0] + 0.1, 2e-5, f"rc = {rc[0]:.2f}'", rotation='vertical', ha='left')
ax.axvline(rc[0], zorder=0, lw=1)
ax.semilogy()

if ylims[0] < 1e-5:
    ax.set_ylim(1e-5, ylims[1])
else:
    ax.set_ylim(ylims)
ax.set_xlabel('Radius [arcmin]')
ax.set_ylabel('Flux [cnts/s]')
ax.legend(loc='upper right')
#fig.savefig(f'/home/boada/PSZ1_G057.42-10.77_radproffit_2.png', bbox='tight', dpi=180)